<a href="https://colab.research.google.com/github/peterbabulik/ZNE/blob/main/ZNE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install mitiq qiskit-ibm-runtime --upgrade -q

In [5]:
import time
import numpy as np
from qiskit.circuit.library import EfficientSU2
from qiskit.quantum_info import SparsePauliOp
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2 as Estimator, EstimatorOptions

# --- 1. CREDENTIALS & CONFIGURATION ---
API_KEY = "WA3o-46Us4d5wTaNYqKcNMyUO8Z5Lk1onIingshloByM"
BACKEND_NAME = "ibm_torino"
NUM_QUBITS = 50
REPS = 1

def main():
    print(f"--- PREPARING QUANTUM ADVANTAGE EXPERIMENT ---")
    print(f"Target: {BACKEND_NAME} | Qubits: {NUM_QUBITS}")

    # 1. Authenticate
    try:
        service = QiskitRuntimeService(
            channel="ibm_quantum_platform",
            token=API_KEY
        )
    except Exception as e:
        print(f"CRITICAL AUTH ERROR: {e}")
        return

    # 2. Select Backend
    try:
        backend = service.backend(BACKEND_NAME)
        print(f"Connected to {backend.name}. Status: {backend.status().state}")
    except:
        print(f"{BACKEND_NAME} unavailable. Finding best available system...")
        try:
            backend = service.least_busy(operational=True, simulator=False)
            print(f"Fallback: {backend.name}")
        except:
            print("No quantum computers available.")
            return

    # 3. Create the "Geometric Shape" (The Circuit)
    print(f"Constructing {NUM_QUBITS}-Qubit Circuit...")
    # Using the class directly (ignoring deprecation warning for simplicity as it works)
    psi = EfficientSU2(num_qubits=NUM_QUBITS, reps=REPS, entanglement="linear")
    psi.measure_all()

    # 4. Define the Observable
    observables = [SparsePauliOp("Z" * NUM_QUBITS)]

    # 5. Transpile
    print("Transpiling to Physical Qubits...")
    pm = generate_preset_pass_manager(backend=backend, optimization_level=3)
    isa_circuit = pm.run(psi)
    isa_observables = [obs.apply_layout(isa_circuit.layout) for obs in observables]

    # 6. Configure Error Mitigation (UPDATED FOR V2)
    options = EstimatorOptions()
    # Enable ZNE (Zero Noise Extrapolation) Explicitly
    options.resilience.zne_mitigation = True
    options.default_shots = 4096

    print("\n--- SUBMITTING JOB TO IBM QUANTUM ---")
    print("Method: EstimatorV2 with ZNE (Explicit)")

    # FIX: Use 'mode' instead of 'backend' for V2 - Corrected based on new Qiskit Runtime V2 usage
    estimator = service.estimator(options=options)

    start_time = time.time()
    try:
        # Submit the job
        job = estimator.run([(isa_circuit, isa_observables)], backend=backend)
        print(f"Job ID: {job.job_id()}")
        print("Waiting for results (Queue time can be 10min - 2 hours)...")

        result = job.result()
    except Exception as e:
        print(f"Job Failed: {e}")
        return

    end_time = time.time()

    # 7. EXTRACT DATA
    # Result structure for V2 is a PrimitiveResult object
    pub_result = result[0]
    exp_val = pub_result.data.evs[0]
    std_error = pub_result.data.stds[0]

    runtime = end_time - start_time
    total_gates = isa_circuit.size()

    print("\n" + "="*40)
    print("      DATA FOR GITHUB TRACKER")
    print("="*40)
    print(f"Expectation Value:   {exp_val}")
    print(f"Standard Error (+/-):{std_error}")
    print(f"Total Gates:         {total_gates}")
    print(f"Qubits:              {NUM_QUBITS}")
    print(f"Backend:             {backend.name}")
    print(f"Runtime (s):         {runtime:.2f}")
    print(f"Method:              EstimatorV2 (ZNE Mitigation)")
    print("="*40)

if __name__ == "__main__":
    main()

qiskit_runtime_service._discover_account:WARNING:2026-01-28 22:40:09,115: Loading account with the given token. A saved account will not be used.


--- PREPARING QUANTUM ADVANTAGE EXPERIMENT ---
Target: ibm_torino | Qubits: 50


qiskit_runtime_service.__init__:WARNING:2026-01-28 22:40:12,710: Instance was not set at service instantiation. Free and trial plan instances will be prioritized. Based on the following filters: (tags: None, region: us-east, eu-de), and available plans: (open), the available account instances are: open-instance. If you need a specific instance set it explicitly either by using a saved account with a saved default instance or passing it in directly to QiskitRuntimeService().
qiskit_runtime_service.backends:WARNING:2026-01-28 22:40:12,711: Using instance: open-instance, plan: open
qiskit_runtime_service.backends:WARNING:2026-01-28 22:40:13,569: Loading instance: open-instance, plan: open


ibm_torino unavailable. Finding best available system...


qiskit_runtime_service.backends:WARNING:2026-01-28 22:40:14,818: Using instance: open-instance, plan: open
/tmp/ipython-input-1501905344.py:44: DeprecationWarning: The class ``qiskit.circuit.library.n_local.efficient_su2.EfficientSU2`` is deprecated as of Qiskit 2.1. It will be removed in Qiskit 3.0. Use the function qiskit.circuit.library.efficient_su2 instead.
  psi = EfficientSU2(num_qubits=NUM_QUBITS, reps=REPS, entanglement="linear")


Fallback: ibm_marrakesh
Constructing 50-Qubit Circuit...
Transpiling to Physical Qubits...

--- SUBMITTING JOB TO IBM QUANTUM ---
Method: EstimatorV2 with ZNE (Explicit)


AttributeError: 'QiskitRuntimeService' object has no attribute 'estimator'

In [8]:
import time
import numpy as np
from qiskit.circuit.library import EfficientSU2
from qiskit.quantum_info import SparsePauliOp
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2 as Estimator, EstimatorOptions

# --- 1. CREDENTIALS & CONFIGURATION ---
API_KEY = "your api key here"
BACKEND_NAME = "ibm_torino"
NUM_QUBITS = 50
REPS = 1

def main():
    print(f"--- PREPARING QUANTUM ADVANTAGE EXPERIMENT ---")
    print(f"Target: {BACKEND_NAME} | Qubits: {NUM_QUBITS}")

    # 1. Authenticate
    try:
        service = QiskitRuntimeService(
            channel="ibm_quantum_platform",
            token=API_KEY
        )
    except Exception as e:
        print(f"CRITICAL AUTH ERROR: {e}")
        return

    # 2. Select Backend
    try:
        backend = service.backend(BACKEND_NAME)
        print(f"Connected to {backend.name}. Status: {backend.status().status_msg}")
    except Exception:
        print(f"{BACKEND_NAME} unavailable. Finding best available system...")
        try:
            backend = service.least_busy(operational=True, simulator=False)
            print(f"Fallback: {backend.name}")
        except Exception:
            print("No quantum computers available.")
            return

    # 3. Create the Circuit
    print(f"Constructing {NUM_QUBITS}-Qubit Circuit...")
    # EfficientSU2 creates a parameterized circuit
    psi = EfficientSU2(num_qubits=NUM_QUBITS, reps=REPS, entanglement="linear")
    psi.measure_all()

    # 4. Define the Observable
    observables = [SparsePauliOp("Z" * NUM_QUBITS)]

    # 5. Transpile
    print("Transpiling to Physical Qubits...")
    pm = generate_preset_pass_manager(backend=backend, optimization_level=3)
    isa_circuit = pm.run(psi)
    isa_observables = [obs.apply_layout(isa_circuit.layout) for obs in observables]

    # 6. Configure Error Mitigation
    options = EstimatorOptions()
    options.resilience.zne_mitigation = True
    options.default_shots = 4096

    print("\n--- SUBMITTING JOB TO IBM QUANTUM ---")
    print("Method: EstimatorV2 with ZNE (Explicit)")

    # FIX: Instantiate Estimator directly
    try:
        estimator = Estimator(mode=backend, options=options)
    except Exception as e:
        print(f"Error initializing Estimator: {e}")
        return

    # --- NEW FIX: GENERATE PARAMETER VALUES ---
    # The circuit has parameters (angles) that must be set.
    # We generate random numbers between 0 and 2*pi for all 200 parameters.
    num_params = isa_circuit.num_parameters
    print(f"Circuit has {num_params} parameters. Generating random values...")

    # Generate random values
    parameter_values = np.random.uniform(0, 2*np.pi, num_params)

    start_time = time.time()
    try:
        # Submit the job with the parameter values included
        # Format: (circuit, observables, parameter_values)
        job = estimator.run([(isa_circuit, isa_observables, parameter_values)])

        print(f"Job ID: {job.job_id()}")
        print("Waiting for results (Queue time can be 10min - 2 hours)...")

        result = job.result()
    except Exception as e:
        print(f"Job Failed: {e}")
        return

    end_time = time.time()

    # 7. EXTRACT DATA
    pub_result = result[0]
    exp_val = pub_result.data.evs[0]
    std_error = pub_result.data.stds[0]

    runtime = end_time - start_time
    total_gates = isa_circuit.size()

    print("\n" + "="*40)
    print("      DATA FOR GITHUB TRACKER")
    print("="*40)
    print(f"Expectation Value:   {exp_val}")
    print(f"Standard Error (+/-):{std_error}")
    print(f"Total Gates:         {total_gates}")
    print(f"Qubits:              {NUM_QUBITS}")
    print(f"Backend:             {backend.name}")
    print(f"Runtime (s):         {runtime:.2f}")
    print(f"Method:              EstimatorV2 (ZNE Mitigation)")
    print("="*40)

if __name__ == "__main__":
    main()

qiskit_runtime_service._discover_account:WARNING:2026-01-28 22:47:28,434: Loading account with the given token. A saved account will not be used.


--- PREPARING QUANTUM ADVANTAGE EXPERIMENT ---
Target: ibm_torino | Qubits: 50


qiskit_runtime_service.__init__:WARNING:2026-01-28 22:47:30,676: Instance was not set at service instantiation. Free and trial plan instances will be prioritized. Based on the following filters: (tags: None, region: us-east, eu-de), and available plans: (open), the available account instances are: open-instance. If you need a specific instance set it explicitly either by using a saved account with a saved default instance or passing it in directly to QiskitRuntimeService().
qiskit_runtime_service.backends:WARNING:2026-01-28 22:47:30,677: Using instance: open-instance, plan: open


Connected to ibm_torino. Status: active
Constructing 50-Qubit Circuit...
Transpiling to Physical Qubits...


/tmp/ipython-input-2626606824.py:44: DeprecationWarning: The class ``qiskit.circuit.library.n_local.efficient_su2.EfficientSU2`` is deprecated as of Qiskit 2.1. It will be removed in Qiskit 3.0. Use the function qiskit.circuit.library.efficient_su2 instead.
  psi = EfficientSU2(num_qubits=NUM_QUBITS, reps=REPS, entanglement="linear")



--- SUBMITTING JOB TO IBM QUANTUM ---
Method: EstimatorV2 with ZNE (Explicit)
Circuit has 200 parameters. Generating random values...
Job ID: d5t9414bmr9c739mr420
Waiting for results (Queue time can be 10min - 2 hours)...

      DATA FOR GITHUB TRACKER
Expectation Value:   0.0001012587895934192
Standard Error (+/-):0.00907142081105357
Total Gates:         794
Qubits:              50
Backend:             ibm_torino
Runtime (s):         22.85
Method:              EstimatorV2 (ZNE Mitigation)
